In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
df_scopus = pd.read_excel('/Users/eric/Downloads/ext_list_October_2020.xlsx')

In [3]:
r = requests.get("https://service.elsevier.com/app/answers/detail/a_id/15181/supporthub/scopus/")
soup = BeautifulSoup(r.text, 'lxml')

asjc = []
asjc_dict = {}
for tr in soup.findAll('tr')[1:]:
    tds = tr.findAll('td')
    code = tds[0].text.strip()
    field = tds[1].text.strip()
    subject_area = tds[2].text.strip()
    elem = {
        "code_asjc": code,
        "field": field,
        "subject_area": subject_area
    }
    asjc_dict[code] = elem
    asjc.append(elem)

In [4]:
data_asjc = []
for i, row in df_scopus[['Print-ISSN', 'E-ISSN', 'All Science Journal Classification Codes (ASJC)']].iterrows():
    
    codes_raw = [c.strip() for c in re.split(';|,',row['All Science Journal Classification Codes (ASJC)'])]
    codes=[]
    for c in codes_raw:
        if len(c) == 0:
            continue
        if c not in asjc_dict:
            print(f"{c} missing")
            continue
        codes.append(asjc_dict[c])
    issns = []
    issns.append(row['Print-ISSN'])
    issns.append(row['E-ISSN'])
    issns = [str(i) for i in issns if not pd.isnull(i)]
    for i in issns:
        if len(i) != 8:
            print(f"weird issn {i}")
        else:
            new_line = {
                "issn": i[0:4]+'-'+i[4:9]
            }
            new_line['asjc'] = codes
            data_asjc.append(new_line)

3330 missing
2290 missing


In [13]:
json.dump(data_asjc, open('asjc.json', 'w'), indent=2)